In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook


from fastmri_recon.data.oasis_sequences import Masked2DSequence
from fastmri_recon.helpers.evaluate import METRIC_FUNCS, Metrics
from fastmri_recon.helpers.reconstruction import zero_filled_recon

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.3.0     installed
numpy          : >=1.16.4  - required | 1.16.4    installed
matplotlib     : >=3.0.0   - required | 3.1.0     installed
astropy        : >=3.0.0   - required | 2.0.8     installed
nibabel        : >=2.3.2   - required | 2.4.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.3.3     installed
scikit-learn   : >=0.19.1  - requi

In [3]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/OASIS_data/'

In [5]:
AF = 4

train_gen = Masked2DSequence(
    train_path, 
    af=AF, 
    inner_slices=32, 
    scale_factor=1e-2, 
    seed=0, 
    val_split=0.1,
)
val_gen = train_gen.val_sequence

val_gen.filenames = val_gen.filenames[:100]

In [6]:
def metrics():
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
        (zero_filled_recon(np.squeeze(val_gen[i][0][0])), np.squeeze(val_gen[i][1]))
        for i in tqdm_notebook(range(len(val_gen)), desc='Val files')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Stats'):
        metrics.push(images, im_recos)
    return metrics

In [7]:
m = metrics()

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:138: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)


In [8]:
m

PSNR = 26.15 +/- 3.564 SSIM = 0.6716 +/- 0.07759